In [10]:
import numpy as np

def test_angle_evaluation(state, evaluation_multiplier, alpha, beta):
    """
    Test function to isolate and debug the angle portion of the evaluation function.
    """
    car_state = state[0]
    car_pos = car_state[0:2]
    car_yaw = car_state[2]

    ooi_state = state[1]
    ooi_reshaped = np.reshape(ooi_state, (4, 2))

    cov_state = state[2]
    cov_diag = np.diag(cov_state)
    corner_traces = np.zeros((int(cov_diag.shape[0]/2),))

    for i in range(corner_traces.shape[0]):
        corner_traces[i] = cov_diag[i*2] + cov_diag[(i*2) + 1]

    cum_bearing = 0.

    for i in range(corner_traces.shape[0]):
        ooi_bearing = np.arctan2(ooi_reshaped[i, 1] - car_pos[1], ooi_reshaped[i, 0] - car_pos[0])
        bearing_delta = abs(ooi_bearing - car_yaw)

        cum_bearing += corner_traces[i] * bearing_delta

        print(f"Corner {i} - Bearing: {bearing_delta}, Covariance Trace: {corner_traces[i]}")
        print(f"Car yaw: {car_yaw}, OOI bearing: {ooi_bearing}")
        if bearing_delta < 0:
            print("Negative bearing delta")
            print(f'Bearing delta: {bearing_delta}')
            exit()

    return evaluation_multiplier * beta * cum_bearing

# Example state input for testing
state = [
    np.array([-5.0, 1.0, np.pi-0.2]),  # Car state [x, y, yaw]
    np.zeros((8,)),  # OOI state (flattened 4 corners)
    np.diag(np.ones((8,)))  # Covariance state (flattened)
]

evaluation_multiplier = 1.0
alpha = 0.5
beta = 0.5

bearing_result = test_angle_evaluation(state, evaluation_multiplier, alpha, beta)
print(f"Cumulative Bearing Result: {bearing_result}")


Corner 0 - Bearing: 3.138988213439674, Covariance Trace: 2.0
Car yaw: 2.941592653589793, OOI bearing: -0.19739555984988075
Corner 1 - Bearing: 3.138988213439674, Covariance Trace: 2.0
Car yaw: 2.941592653589793, OOI bearing: -0.19739555984988075
Corner 2 - Bearing: 3.138988213439674, Covariance Trace: 2.0
Car yaw: 2.941592653589793, OOI bearing: -0.19739555984988075
Corner 3 - Bearing: 3.138988213439674, Covariance Trace: 2.0
Car yaw: 2.941592653589793, OOI bearing: -0.19739555984988075
Cumulative Bearing Result: 12.555952853758695
